# Q-Learning Case Study: Elevator Choosing Floors

**Analogy:** Imagine an elevator learning which floor to go to. Here, we simulate with FrozenLake-v1.

In [6]:
import numpy as np
import gymnasium as gym

env = gym.make('FrozenLake-v1', is_slippery=False)
n_states = env.observation_space.n
n_actions = env.action_space.n
Q = np.zeros((n_states, n_actions))

# alpha, gamma = 0.1, 0.99
# epsilon, eps_min, eps_decay = 1.0, 0.01, 0.995
returns = []

In [7]:
n_episodes = 20000   # more training
alpha = 0.8          # stronger updates
gamma = 0.95         # discount
epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.999

In [8]:
def epsilon_greedy_action(Q, state, epsilon, action_space):
    return action_space.sample() if np.random.rand() < epsilon else int(np.argmax(Q[state]))

In [9]:
for ep in range(n_episodes):
    state, _ = env.reset()
    done = False
    while not done:
        action = epsilon_greedy_action(Q, state, epsilon, env.action_space)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
        state = next_state
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

# Evaluation
successes = 0
for _ in range(500):
    s,_=env.reset()
    done=False
    while not done:
        a = np.argmax(Q[s])
        s,r,terminated,truncated,_ = env.step(a)
        done=terminated or truncated
        successes+=r
print("Success rate over 500 eval episodes:", successes/500)

Success rate over 500 eval episodes: 1.0


In [19]:
# import time

# env = gym.make("FrozenLake-v1", is_slippery=False, render_mode="human")  # for local machine
# state, _ = env.reset()

# for _ in range(50):   # play 50 steps
#     action = np.argmax(Q[state])  # follow learned policy
#     state, reward, terminated, truncated, info = env.step(action)
#     time.sleep(0.5)   # slow down for visibility
#     if terminated or truncated:
#         state, _ = env.reset()
#         time.sleep(2)

# env.close()

# DQN Case Study: Playing Atari

Analogy: Computer learns to play Atari. Here simulated with CartPole.

In [21]:
!pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 5.4 MB/s eta 0:00:00


In [22]:
import gymnasium as gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

env = gym.make('CartPole-v1')
model = DQN('MlpPolicy', env, learning_rate=1e-3, buffer_size=50000,
            exploration_fraction=0.1, exploration_final_eps=0.02, verbose=1)
model.learn(total_timesteps=20000)
mean,std=evaluate_policy(model,env,n_eval_episodes=10)
print('DQN Eval:',mean,std)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 23       |
|    ep_rew_mean      | 23       |
|    exploration_rate | 0.955    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 11747    |
|    time_elapsed     | 0        |
|    total_timesteps  | 92       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 21.1     |
|    ep_rew_mean      | 21.1     |
|    exploration_rate | 0.917    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1263     |
|    time_elapsed     | 0        |
|    total_timesteps  | 169      |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.328    |
|    n_updates        | 17       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean    

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Streaming output truncated to the last 5000 lines.
|    learning_rate    | 0.001    |
|    loss             | 0.000433 |
|    n_updates        | 1271     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.1     |
|    ep_rew_mean      | 10.1     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 420      |
|    fps              | 1407     |
|    time_elapsed     | 3        |
|    total_timesteps  | 5229     |
| train/              |          |
|    learning_rate    | 0.001    |
|    loss             | 0.000257 |
|    n_updates        | 1282     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 10.1     |
|    ep_rew_mean      | 10.1     |
|    exploration_rate | 0.02     |
| time/               |          |
|    episodes         | 424      |
|    fps              | 1404     |
|   

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [ ]:
# import time

# env = gym.make("CartPole-v1", render_mode="human")  # needs render_mode
# obs, _ = env.reset()

# for _ in range(500):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     time.sleep(0.02)   # smooth animation
#     if terminated or truncated:
#         obs, _ = env.reset()

# PPO Case Study: Robot Arm Control

Analogy: A robot arm must move smoothly. PPO ensures small updates.

In [23]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

env=gym.make('Pendulum-v1')
model=PPO('MlpPolicy',env,verbose=1)
model.learn(total_timesteps=50000)
mean,std=evaluate_policy(model,env,n_eval_episodes=5)
print('PPO Eval:',mean,std)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.14e+03 |
| time/              |           |
|    fps             | 1899      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 200        |
|    ep_rew_mean          | -1.12e+03  |
| time/                   |            |
|    fps                  | 884        |
|    iterations           | 2          |
|    time_elapsed         | 4          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00219273 |
|    clip_fraction        | 0.00732    |
|    clip_range           | 0.2        |
|    entropy_loss      

In [ ]:
# env = gym.make("Pendulum-v1", render_mode="human")
# obs, _ = env.reset()

# for _ in range(500):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     if terminated or truncated:
#         obs, _ = env.reset()


# A2C Case Study: Self-Driving Cars

Analogy: Actor (driver) and Critic (coach) for safe driving. Simulated with LunarLander.

In [27]:
!pip install swig
!pip install gymnasium[box2d]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp312-cp312-linux_x86_64.whl size=2409499 sha256=f39569015d08ea9b44036bbfc0bf1ad2408f969dda8c2a877b60a221961c7cb8
  Stored in directory: /root/.cache/pip/wheels/2a/e9/60/774da0bcd07f7dc7761a8590fa2d065e4069568e78dcdc3318
Successfully built box2d-py


In [28]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy

env=gym.make('LunarLander-v3')
model=A2C('MlpPolicy',env,verbose=1)
model.learn(total_timesteps=50000)
mean,std=evaluate_policy(model,env,n_eval_episodes=5)
print('A2C Eval:',mean,std)

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 88.4     |
|    ep_rew_mean        | -342     |
| time/                 |          |
|    fps                | 804      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.38    |
|    explained_variance | 0.000633 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -17.1    |
|    value_loss         | 169      |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 91.6     |
|    ep_rew_mean        | -324     |
| time/                 |          |
|    fps                | 839      |
|    iterations         | 200      |
|    time_elapsed 

/usr/local/lib/python3.12/dist-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


A2C Eval: 157.85720770067437 93.98855303882262


In [ ]:
# env = gym.make("LunarLander-v3", render_mode="human")
# obs, _ = env.reset()

# for _ in range(1000):
#     action, _ = model.predict(obs, deterministic=True)
#     obs, reward, terminated, truncated, info = env.step(action)
#     if terminated or truncated:
#         obs, _ = env.reset()
